In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard
import random

# Third Party
from datasets import Dataset, load_dataset
from openai import OpenAI
from rich import print
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax
import yaml

# First Party
from sdg_hub.flow import Flow
from sdg_hub.sdg import SDG
from blocks import *

/home/lab/.conda/envs/sdg_hub/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 🧾 Use Case 3: Structured Summary from Conversations

In many enterprise workflows, unstructured conversations—like meeting transcripts, support calls, or financial briefings—contain valuable insights that need to be distilled into structured formats for downstream processing.

In this use case, we aim to extract **structured summaries** from raw conversational transcripts. Specifically, we want the model to extract:

- A concise **summary** of the conversation
- Relevant **keywords**
- **Named entities** such as people, organizations, and dates
- The **sentiment** of the discussion


### Why is this useful?

Rather than just generating freeform text, this task helps transform unstructured inputs into **machine-readable structured outputs**, making it suitable for:

- Reporting dashboards  
- Automated indexing and retrieval  
- Analytics pipelines  
- Compliance and auditing tools

### PDF Data 

You can find the pdf data here: [seed_data/financial_call_transcripts](seed_data/financial_call_transcripts)

## 🧑‍🏫 Step 1: Set Up the Teacher Model

This demo expects an openai compatible endpoint. You can use your favorite inference server like vLLM, HFInferenceServer, LlamaStack, etc. For more details on how to setup an inference server using vLLM, please refer to the [README](README.md).

For this demo we will use Llama-3.3-70B-Instruct as our teacher model.

#### Let's test the connection

In [3]:
openai_api_key = "EMPTY"  # replace with your inference server api key
openai_api_base = (
    "http://0.0.0.0:8000/v1"  # replace with your inference server endpoint
)


client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
teacher_model = models.data[0].id

# Test the connection with a simple completion
response = client.chat.completions.create(
    model=teacher_model,
    messages=[{"role": "user", "content": "Hello!"}],
    temperature=0.0,
    max_tokens=10,
)
completion = response.choices[0].message.content

print(f"Connection successful! {teacher_model}: {completion}")

[19:30:15] INFO     HTTP Request: GET http://0.0.0.0:8000/v1/models "HTTP/1.1 200 OK"               ]8;id=589681;file:///home/lab/.conda/envs/sdg_hub/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=528115;file:///home/lab/.conda/envs/sdg_hub/lib/python3.11/site-packages/httpx/_client.py#1025\1025]8;;\

[19:30:16] INFO     HTTP Request: POST http://0.0.0.0:8000/v1/chat/completions "HTTP/1.1 200 OK"    ]8;id=835684;file:///home/lab/.conda/envs/sdg_hub/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=977004;file:///home/lab/.conda/envs/sdg_hub/lib/python3.11/site-packages/httpx/_client.py#1025\1025]8;;\

Connection successful! meta-llama/Llama-3.3-70B-Instruct: Hello. How can I help you today?

In [4]:
seed_data = load_dataset(
    "json", data_files="seed_data/financial_call_transcripts.jsonl", split="train"
)

seed_data[0]

{'conversation_id': 'c47a92e006b54d014a79b447528c55a7',
 'pdf_path': 'seed_data/financial_call_transcripts/c47a92e006b54d014a79b447528c55a7.pdf'}

## Setting up the pipeline


In this section, we walk through a pipeline designed to process financial call transcripts (in PDF format) and extract structured insights using LLM-powered blocks. This is a classic example of transforming unstructured text into structured JSON for downstream analysis.



This YAML defines a flow that reads PDF transcripts, parses them into text, and then uses LLMs to extract:
* ✅ Summary of the transcript
* ✅ Key topics or keywords
* ✅ Mentioned named entities (people, organizations, locations, etc.)
* ✅ Overall sentiment of the call

All of these are combined into a final structured json_output.


```mermaid 
graph LR
    A[PDF Transcript] --> B[parse_transcript<br/>DoclingParsePDF]
    B --> C[conversation]
    C --> D[add_question<br/>AddStaticValue]
    D --> E[gen_summary<br/>LLMBlock]
    E --> F[gen_keywords<br/>LLMBlock]
    F --> G[gen_named_entities<br/>LLMBlock]
    G --> H[gen_sentiment<br/>LLMBlock]

    H --> I[format_json<br/>JSONFormat]
    I --> J[json_output]
```

In [5]:
# Lets look at the skills flow we will be using
# flows/grounded_summary_extraction.yaml

with open("flows/grounded_summary_extraction.yaml", "r") as f:
    flow_config = yaml.safe_load(f)

# Convert to YAML string with indentation for pretty printing
yaml_str = yaml.dump(flow_config, indent=2, default_flow_style=False, sort_keys=False)

# Create syntax highlighted YAML
syntax = Syntax(yaml_str, "yaml", theme="github")

# Display it inside a panel with a descriptive title
console = Console()
console.print(Panel(
    syntax, 
    title="🔧 Flow Configuration: Grounded Summary Extraction", 
    subtitle="📋 Pipeline for extracting structured insights from PDF transcripts",
))

╭────────────────────────────── 🔧 Flow Configuration: Grounded Summary Extraction ───────────────────────────────╮
│ - block_type: DoclingParsePDF                                                                                   │
│   block_config:                                                                                                 │
│     block_name: parse_transcript                                                                                │
│     pdf_path_column: pdf_path                                                                                   │
│     output_column: conversation                                                                                 │
│ - block_type: AddStaticValue                                                                                    │
│   block_config:                                                                                                 │
│     block_name: add_question                                                                                    │
│     column_name: question                                                                                       │
│     static_value: Extract summary, keywords, named entities, and sentiment from the                             │
│       transcript and return in JSON format.                                                                     │
│ - block_type: LLMBlock                                                                                          │
│   block_config:                                                                                                 │
│     block_name: gen_summary                                                                                     │
│     model_id: meta-llama/Llama-3.3-70B-Instruct                                                                 │
│     config_path: ../prompts/summary.yaml                                                                        │
│     output_cols:                                                                                                │
│     - summary                                                                                                   │
│ - block_type: LLMBlock                                                                                          │
│   block_config:                                                                                                 │
│     block_name: gen_keywords                                                                                    │
│     model_id: meta-llama/Llama-3.3-70B-Instruct                                                                 │
│     config_path: ../prompts/keywords.yaml                                                                       │
│     output_cols:                                                                                                │
│     - keywords                                                                                                  │
│ - block_type: LLMBlock                                                                                          │
│   block_config:                                                                                                 │
│     block_name: gen_named_entities                                                                              │
│     model_id: meta-llama/Llama-3.3-70B-Instruct                                                                 │
│     config_path: ../prompts/named_entities.yaml                                                                 │
│     output_cols:                                                                                                │
│     - named_entities                                                                                            │
│ - block_type: LLMBlock                                                                                          │
│   block_config:                                        

In [6]:
# Load the flow
flow = Flow(client).get_flow_from_file("flows/grounded_summary_extraction.yaml")

# Initialize the synthetic data generator
generator = SDG(flows=[flow])

In [ ]:
seed_data = seed_data.select(range(10)) # note: this is just for demo purposes, in practice you can use the entire dataset
generated_data = generator.generate(seed_data)

In [10]:
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax
import random
import json


rand_idx = random.choice(range(len(generated_data)))
# Your data
data = json.loads(generated_data[rand_idx]["json_output"])

# Convert to JSON string with indentation for pretty printing
json_str = json.dumps(data, indent=2)

# Create syntax highlighted JSON
syntax = Syntax(json_str, "json", theme="github", line_numbers=False)

# Display it inside a panel
console = Console()
console.print(Panel(syntax, title="📊 Extracted Summary", expand=True))

╭───────────────────────────────────────────── 📊 Extracted Summary ──────────────────────────────────────────────╮
│ {                                                                                                               │
│   "summary": "American Airlines Group reported a strong third quarter despite weather and operational challenge │
│   "keywords": [                                                                                                 │
│     "American Airlines",                                                                                        │
│     "Q3 earnings",                                                                                              │
│     "revenue outlook",                                                                                          │
│     "Premium Economy",                                                                                          │
│     "Basic Economy",                                                                                            │
│     "AAdvantage program",                                                                                       │
│     "capacity management",                                                                                      │
│     "fuel prices",                                                                                              │
│     "guidance",                                                                                                 │
│     "shareholder value"                                                                                         │
│   ],                                                                                                            │
│   "named_entities": {                                                                                           │
│     "organizations": [                                                                                          │
│       "American Airlines Group",                                                                                │
│       "NAACP"                                                                                                   │
│     ],                                                                                                          │
│     "people": [                                                                                                 │
│       "Derek",                                                                                                  │
│       "Robert"                                                                                                  │
│     ],                                                                                                          │
│     "locations": [                                                                                              │
│       "Miami",                                                                                                  │
│       "Florida",                                                                                                │
│       "Puerto Rico",                                                                                            │
│       "Caribbean",                                                                                              │
│       "Mexico City"                                                                                             │
│     ],                                                                                                          │
│     "dates": [                                                                                                  │
│       "Third Quarter 2017",                                                                                     │
│       2018,                                                                                                     │
│       "September 5",                                   

## ✅ Conclusion

In this section, we demonstrated how to construct an end-to-end pipeline that transforms unstructured PDF transcripts into structured JSON insights using modular building blocks. By parsing the document, prompting an LLM to extract specific features, and formatting the results, we’ve created a scalable workflow for financial document analysis—or any use case involving long-form text.

## 📝 Homework: Extend the Pipeline

Your task is to add a new block that extracts a different kind of structured insight from the conversation context.

Some examples include:
* 🧩 Risk factors mentioned in the call
* 📊 Numerical metrics (e.g., revenue, margin, headcount)
* 📌 Action items or decisions discussed

Once you’ve done that, you’ll have taken the first step toward custom skill authoring, opening the door to richer document understanding tailored to your own domain needs. 